$$\huge{\text{Data Science Workshop}}$$

This notebook serves as an hands-on introduction to the data science pipeline.  Using a single dataset throughout, it begins with loading the data from a Greenplum Database (GPDB), then proceeds to data exploration, feature engineering, model development, and finally, model evaluation.

We’ll be using the publicly available [Abalone dataset from the University of California, Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/abalone).  The dataset contains eight attributes (including our target prediction column).

| Column Name | Data Type | Description|
| ---|:---:| ---:|
|Sex | text | M,F,I[infant]|
| Length | float | Longest shell measurement|
|Diameter | float | Perpendicular to length|
| Height | float | With meat in shell |
| Whole weight | float | Whole abalone |
| Shucked weight | float | Weight of meat only |
| Viscera weight | float | Gut weight (after bleeding) |
| Shell weight | float | Post-drying |
| Rings | integer | +1.5 gives the age in years|

Much of the code to conduct this enterprise has already been filled in for you.  Where your input is required will be clear as it will say “*code here*“.  You’ll replace those sections with the appropriate code snippets you learn as we go through this notebook together.  You should feel free to make as many comments and notes as is helpful for your future self (you can make an inline comment by beginning a line with the “#” character).

# Connect to Database

Establishing the sql connection and loading the data into the GPDB is done behind the scenes here through the call to the `___` function in the interest in getting more quickly to the sections on analytics.

In [ ]:
%matplotlib inline

In [ ]:
import dbconnect

In [ ]:
db_credential_file = '../.dbcred'
dbconnect.connect_and_register_sql_magic(db_credential_file)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
pd.set_option('display.max_columns', 200)

## Load Data: Abalone

In [ ]:
abalone_columns = (
    'sex',
    'length',
    'diameter',
    'height',
    'whole_weight',
    'shucked_weight',
    'viscera_weight',
    'shell_weight',
    'rings'
)
len(abalone_columns)

In [ ]:
df_abalone = pd.read_csv('../input/abalone.data', names=abalone_columns)

In [ ]:
df_abalone.info()

In [ ]:
df_abalone['sex'] = df_abalone['sex'].str.lower()

In [ ]:
(df_abalone.rings + 1.5).hist(bins=30)

In [ ]:
((df_abalone.rings + 1.5) >= 3).value_counts()

In [ ]:
_cumsum = (df_abalone.rings + 1.5).value_counts().sort_index().cumsum()
_cumsum / df_abalone.shape[0]

In [ ]:
engine = create_engine(
    'postgresql://{user}:{password}@{host}:{port}/{database}'.format(
        **dbconnect.fetchDBCredentials('../.dbcred')
    )
)

In [ ]:
schema = 'ds_training_wnv'

In [ ]:
%read_sql DROP SCHEMA IF EXISTS {schema} CASCADE;
%read_sql CREATE SCHEMA {schema};

In [ ]:
df_abalone.to_sql(
    'abalone', 
    engine, 
    schema=schema, 
    if_exists='replace', 
    index=True, 
    index_label='id',
    chunksize=10000)

In [ ]:
%%read_sql
SELECT *
FROM {schema}.abalone
LIMIT 10

# Data pre-processing

## Define target (age)

Our first order of business is to generate our prediction target.  This is a two step process. We’ll create a new column in our data table (“age”) by adding 1.5 to the “rings” column to generate the abalone age.  We’ll then create another column (“mature”) denoting abalone maturity as either a 1 or 0 based whether the age is greater than or equal to an age of 10 years.

In [ ]:
%%read_sql
DROP TABLE IF EXISTS {schema}.abalone_target;
CREATE TABLE {schema}.abalone_target
AS
SELECT 
    *,
    rings + 1.5 as age,
    CASE WHEN 
            (rings + 1.5) >= 10.0
        THEN 1
        ELSE 0
    END as mature
FROM {schema}.abalone

In [ ]:
%%read_sql
SELECT *
FROM {schema}.abalone_target 
LIMIT 10

In [ ]:
%%read_sql
SELECT sum(mature), count(*)
FROM {schema}.abalone_target 


## Encode categorical variables

The next thing is to leverage [MADlib to one-hot encode](http://madlib.apache.org/docs/latest/group__grp__encode__categorical.html) the “sex” column which is a categorical variable.  In order to create a predictive model, we need all our columns to be numerical values.  Making sure all our model inputs conform to this standard is an important part of the data science modeling pipeline and is considered part of the preprocessing/data cleaning step of the process. 

In [ ]:
%%read_sql
SELECT
madlib.encode_categorical_variables (
        '{schema}.abalone_target',  -- input table
        '{schema}.abalone_encoded',  -- output table
        'sex',   -- categorical_cols
        NULL,  --categorical_cols_to_exclude    -- Optional
        NULL,  --row_id,                         -- Optional
        NULL,  --top,                            -- Optional
        NULL,  --value_to_drop,                  -- Optional
        NULL,  --encode_null,                    -- Optional
        NULL,  --output_type,                    -- Optional
        NULL,  --output_dictionary,              -- Optional
        NULL  --distributed_by                  -- Optional
    )

In [ ]:
%%read_sql
SELECT *
FROM {schema}.abalone_encoded
LIMIT 5

# Explore data

In [ ]:
%%read_sql
SELECT
madlib.summary ( 
    '{schema}.abalone_encoded',  -- source_table,
    '{schema}.abalone_summary',  -- output_table,
    NULL,  -- target_cols,
    NULL  -- grouping_cols,
)

In [ ]:
%%read_sql
SELECT *
FROM {schema}.abalone_summary
LIMIT 15

In [ ]:
%%read_sql
SELECT target_column
from {schema}.abalone_summary

In [ ]:
%%read_sql
SELECT *
FROM {schema}.abalone_encoded
limit 3

In [ ]:
%%read_sql
SELECT
madlib.correlation(
    '{schema}.abalone_encoded', -- source_table,
    '{schema}.abalone_correlations', -- output_table,
    'length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings', -- target_cols,
    TRUE, -- verbose,
    'sex_f,sex_i,sex_m'  -- grouping_columns
)

In [ ]:
%%read_sql
SELECT * 
from {schema}.abalone_correlations
ORDER BY sex_m, sex_f, sex_i, column_position

In [ ]:
%%read_sql
SELECT
madlib.train_test_split(
    '{schema}.abalone_encoded', -- source_table,
    '{schema}.abalone_classif', -- output_table,
    0.7, -- train_proportion,
    NULL, -- test_proportion,
    NULL, -- grouping_cols,
    'id,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,sex_f,sex_i,sex_m,mature', -- target_cols,
    FALSE, -- with_replacement,
    TRUE -- separate_output_tables
)

The train/test flag is in column `split`. `1` means train, `0` means test

In [ ]:
%%read_sql
SELECT count(*) as n
FROM {schema}.abalone_classif_train

In [ ]:
%%read_sql
SELECT count(*) as n
FROM {schema}.abalone_classif_test

# Modeling

## Classification

### Logistic Regression

Note: drop one of the 1-hot-encoded variables to remove perfect collinearity

In [ ]:
%%read_sql
SELECT
madlib.logregr_train(
    '{schema}.abalone_classif_train', -- source_table,
    '{schema}.abalone_logreg_model', -- out_table,
    'mature', -- dependent_varname,
    'ARRAY[
        1,
        length,
        diameter,
        height,
        whole_weight,
        shucked_weight,
        viscera_weight,
        shell_weight,
        sex_f,
        sex_m
    ]' -- independent_varname,
    --, -- grouping_cols,
    --, -- max_iter,
    --, -- optimizer,
    --, -- tolerance,
     -- verbose
)

In [ ]:
%%read_sql
SELECT *
FROM {schema}.abalone_logreg_model
LIMIT 5

cross join model with table to be scored

In [ ]:
%%read_sql
CREATE TABLE {schema}.abalone_logreg_test_proba
AS
SELECT madlib.logregr_predict_prob(
        coef, 
        ARRAY[
            1,
            length,
            diameter,
            height,
            whole_weight,
            shucked_weight,
            viscera_weight,
            shell_weight,
            sex_f,
            sex_m
        ] 
    ) as proba,
    test.mature
FROM {schema}.abalone_classif_test test, {schema}.abalone_logreg_model model
;

In [ ]:
%%read_sql
SELECT *
FROM {schema}.abalone_logreg_test_proba
LIMIT 4

In [ ]:
%%read_sql
SELECT
madlib.area_under_roc(
    '{schema}.abalone_logreg_test_proba', -- table_in, 
    '{schema}.abalone_logreg_test_auc',  --table_out,
    'proba',  -- prediction_col, 
    'mature'  --observed_col, 
)

In [ ]:
%%read_sql
SELECT *
FROM {schema}.abalone_logreg_test_auc

In [ ]:
%%read_sql
CREATE TABLE {schema}.abalone_logreg_test_predict
AS
SELECT
    (proba >= 0.5)::integer as predicted,
    mature
FROM {schema}.abalone_logreg_test_proba

In [ ]:
%%read_sql
SELECT
madlib.confusion_matrix(
    '{schema}.abalone_logreg_test_predict', -- table_in
    '{schema}.abalone_logreg_test_conf_matrix', -- table_out
    'predicted',  --prediction_col
    'mature' --observation_col
)

Check by hand which axis corresponds to actual vs predicted

In [ ]:
%%read_sql
SELECT
    count(*)
FROM {schema}.abalone_logreg_test_predict
WHERE
    predicted = 1 AND
    mature = 0


In [ ]:
%%read_sql
SELECT 
    row_id,
    class,
    confusion_arr[1] as predicted_0,
    confusion_arr[2] as predicted_1
FROM {schema}.abalone_logreg_test_conf_matrix
ORDER BY row_id

Get ROC values (thresholds, true-positives, false-positives)

In [ ]:
%%read_sql
SELECT
madlib.binary_classifier(
    '{schema}.abalone_logreg_test_proba', -- table_in
    '{schema}.abalone_logreg_test_binary_metrics', -- table_out
    'proba',  --prediction_col
    'mature' --observation_col
)

In [ ]:
%%read_sql
SELECT *
FROM {schema}.abalone_logreg_test_binary_metrics
WHERE 
    --round(threshold::numeric, 1) = 0.5
    threshold >= 0.48 AND
    threshold <= 0.52
ORDER BY threshold

### Random Forest

In [ ]:
%%read_sql
SELECT * 
FROM {schema}.abalone_classif_train
LIMIT 5

In [ ]:
%%read_sql
SELECT
madlib.forest_train(
    '{schema}.abalone_classif_train',  -- training_table_name
    '{schema}.abalone_rf_model',  -- output_table_name
    'id',  -- id_col_name
    'mature',  -- dependent_variable
    'length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,sex_f,sex_m',  -- list_of_features
    NULL,  -- list_of_features_to_exclude
    NULL,  -- grouping_columns
    10  -- number of trees
)


In [ ]:
%%read_sql
SELECT * 
FROM {schema}.abalone_rf_model
LIMIT 10

In [ ]:
%%read_sql
SELECT
madlib.forest_predict(
    '{schema}.abalone_rf_model',  -- random_forest_model
    '{schema}.abalone_classif_test',  -- new_data_table
    '{schema}.abalone_rf_test_proba',  -- output_table
    'prob'  -- type
)

In [ ]:
%%read_sql
SELECT * 
FROM {schema}.abalone_rf_test_proba
LIMIT 5